In [ ]:
import spacy
import json
import requests
import boto3
import os

In [3]:
CLIENT = boto3.client('dataexchange', region_name = 'us-east-1')
DATA_SET_ID = 'b5c571b26671fc8b73a1146b140f0d75'
REVISION_ID = '9ae2799c1227cc58002a946e27410dd3'
ASSET_ID = '245ba9ec692792d750753ab605f388d2'

In [4]:
nlp = spacy.load("en_core_sci_md")

In [5]:
#finding drug unique number
drugs = ['Amoxicillin','Prinivil']

In [6]:
def unique_number(drugs):
    QUERY_STRING_PARAMETERS = {'search': drugs[0].upper(),'limit': '10' }
    response_one = CLIENT.send_api_asset(
        DataSetId=DATA_SET_ID,
        RevisionId=REVISION_ID,
        AssetId=ASSET_ID,
        Method='GET',
        Path='/drug/drugsfda.json',
        QueryStringParameters=QUERY_STRING_PARAMETERS
    )
    for i in range(10):
        try:
            unique_number_one = json.loads(response_one['Body'])['results'][i]['openfda']['unii'][0]
        except:
            pass
    
    QUERY_STRING_PARAMETERS = {'search': drugs[1].upper(),'limit': '10' }
    response_two = CLIENT.send_api_asset(
        DataSetId=DATA_SET_ID,
        RevisionId=REVISION_ID,
        AssetId=ASSET_ID,
        Method='GET',
        Path='/drug/drugsfda.json',
        QueryStringParameters=QUERY_STRING_PARAMETERS
    )
    for i in range(10):
        try:
            unique_number_two = json.loads(response_one['Body'])['results'][i]['openfda']['unii'][0]
        except:
            pass
    return unique_number_one, unique_number_two

In [7]:
unique_number_one, unique_number_two = unique_number(drugs)

In [8]:
def drug_information(unique_number_one, unique_number_two):
    QUERY_STRING_PARAMETERS_ONE = {'search': unique_number_one,'limit': '1' }
    response_one = CLIENT.send_api_asset(
        DataSetId=DATA_SET_ID,
        RevisionId=REVISION_ID,
        AssetId=ASSET_ID,
        Method='GET',
        Path='/drug/drugsfda.json',
        QueryStringParameters=QUERY_STRING_PARAMETERS_ONE
    )      
    
    QUERY_STRING_PARAMETERS_TWO = {'search': unique_number_two,'limit': '1' }

    response_two = CLIENT.send_api_asset(
        DataSetId=DATA_SET_ID,
        RevisionId=REVISION_ID,
        AssetId=ASSET_ID,
        Method='GET',
        Path='/drug/label.json',
        QueryStringParameters=QUERY_STRING_PARAMETERS_TWO
    )

    return response_one, response_two

In [9]:
response_one, response_two = drug_information(unique_number_one, unique_number_two)

In [10]:
def contradictions(response_one,response_two):
    
    active_ingredients_one = ""
    active_ingredients_two = ""

    try:
        for name in range(len(json.loads(response_one['Body'])['results'][0]['products'][0]['active_ingredients'])):
          active_ingredients_one = json.loads(response_one['Body'])['results'][0]['products'][0]['active_ingredients'][name]['name']
    except:
        pass

    try:
        for name in range(len(json.loads(response_two['Body'])['results'][0]['products'][0]['active_ingredients'])):
          active_ingredients_one = json.loads(response_two['Body'])['results'][0]['products'][0]['active_ingredients'][name]['name']
    except:
        pass

    interactions_one = tuple()
    interactions_two = tuple()
    try:
        drug_interactions_one = json.loads(response_one['Body'])['results'][0]['drug_interactions'][0]
        interactions_one = nlp(drug_interactions_one).ents
    except:
        pass

    try:
        drug_interactions_two = json.loads(response_two['Body'])['results'][0]['drug_interactions'][0]
        interactions_two = nlp(drug_interactions_two).ents
    except:
        pass
    
    res_one = "Some information missing in database "
    res_two = "No hazardous reaction information was found between active components of one drug with the chemical interactions of other drug "
    res_three = "Found chemical interactions between these drugs please consult doctor before consumption "

    if active_ingredients_two=="" and len(interactions_one)==0 and active_ingredients_one=="" and len(interactions_two)==0:
        result = res_one
    if len(list(set(interactions_one).intersection(list(active_ingredients_two))))==0 and len(list(set(interactions_two).intersection(list(active_ingredients_one))))==0:
        result = res_two
    else:
        result = res_three
    
    return result 
  

In [11]:
result = contradictions(response_one,response_two)

In [12]:
result

'No hazardous reaction information was found between active components of one drug with the chemical interactions of other drug '